In [1]:
import matplotlib
from matplotlib import pylab
%config InlineBackend.figure_format = 'retina'
matplotlib.rcParams['figure.dpi'] = 80
params = {'legend.fontsize': 'xx-large',
          'figure.figsize': (6, 4),
         'axes.labelsize': 'xx-large',
         'axes.titlesize': 'xx-large',
         'xtick.labelsize': 'x-large',
         'ytick.labelsize': 'x-large',
         'figure.facecolor': 'w'}
pylab.rcParams.update(params)
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec

import h5py
import numpy as np
import os
import sys
import yaml
from scipy.optimize import curve_fit

%load_ext autoreload
%autoreload 2
sys.path.insert(1, '../code')
from geometric_features import GeometricFeaturizer, geo_name
from scalar_features import ScalarFeaturizer, scalar_name
from read_halos import SimulationReader
from fit import LinearFitter
#from neural_net import NeuralNet, NNFitter
import plotter
import utils
import feature_importance

/ext3/miniconda3/envs/eqenv/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
sim_name = 'TNG100-1'
sim_name_dark = f'{sim_name}-Dark'

halo_tag = ''

In [3]:
fn_halo_config = f'../configs/halos_{sim_name}{halo_tag}.yaml'

with open(fn_halo_config, 'r') as file:
    halo_params = yaml.safe_load(file)

fn_dark_halo_arr = halo_params['halo']['fn_dark_halo_arr']

In [4]:
sim_reader = SimulationReader(halo_params['sim']['base_dir'], sim_name, sim_name_dark, 
                              halo_params['sim']['snap_num_str'])
sim_reader.load_dark_halo_arr(fn_dark_halo_arr)
sim_reader.read_simulations()

In [16]:
sim_reader.add_MAH_to_halos()

0 20299
[    74    155    249 ... 431820 431898 432030]
HalopropIndex_Snapshot
HalopropIndex_Snapshot 0 1346898
HalopropIndex
HalopropIndex 2 2306617
HalopropRootHaloID
HalopropRootHaloID 1122934794 1132041752
HalopropRedshift
HalopropRedshift 0.0 0.0
HalopropFoFIndex_DM
HalopropFoFIndex_DM -1 4226617
19810
count: 0
0 1124718617
[0 1 1]
HalopropOffsets
[1621748, 1863626]
<HDF5 group "/Offsets" (2 members)>
<HDF5 dataset "HalopropOffsets": shape (2306618,), type "<u4">
count: 1
1 1125365056
[1 3 3]
HalopropOffsets
[207769, 609375]
<HDF5 group "/Offsets" (2 members)>
<HDF5 dataset "HalopropOffsets": shape (2207258,), type "<u4">
count: 2
2 1125074886
[1 2 2]
HalopropOffsets
[495034, 623711]
<HDF5 group "/Offsets" (2 members)>
<HDF5 dataset "HalopropOffsets": shape (1925532,), type "<u4">
count: 3
3 1126604720
[0 3 3]
HalopropOffsets
[350158, 543868]
<HDF5 group "/Offsets" (2 members)>
<HDF5 dataset "HalopropOffsets": shape (1789829,), type "<u4">
count: 4
4 1131650284
[4 3 3]
HalopropOff

KeyError: 11

In [ ]:
halo = sim_reader.dark_halo_arr[0]
a_mah, m_mah = halo.catalog_properties['MAH']

In [ ]:
plt.plot(a_mah, m_mah, marker)